# 評価方法

分類されていないデータを認識し、どれだけ正しくカテゴリごとに分類できるかを算出した「平均精度」の高さを競い合います。

今回、活用するデータはLSWMD_25519となります。
LSWMD_25519のFailureType項目が分類されていない状態のデータに対し、正しいFailureTypeカテゴリを分類するプログラムを作成し、その平均精度を算出します。
平均精度とは、カテゴリごとに正しく分類できる精度を平均した値です。カテゴリごとに算出した精度（Aが正しく分類された数/Aのデータ数）を足し、カテゴリ数で割ります。

公平な評価を実施するために、以下の制限を設けています。
1. 外部パッケージをインストールするためのセルとsolution関数の中身のみを編集すること
2. 校舎のiMac上で最後のセルの実行時間が15分未満であること　（%%timeitの出力結果を確認してください）

※気になる点がある場合、Discordで気軽にお問合せください。

In [ ]:
import numpy as np # https://numpy.org/ja/
import pandas as pd # https://pandas.pydata.org/
from sklearn.model_selection import train_test_split

: 

外部パッケージを使用する場合、以下の方法でインストールを実施してください。

In [2]:
# 必要な外部パッケージは、以下の内容を編集しインストールしてください
# !pip install keras
# !pip install tensorflow[and-cuda]

以下のsolution関数のみ編集してください。

In [3]:
import numpy as np
import pandas as pd
from skimage.transform import resize
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, stride=2)
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(64 * 5 * 5, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# def get_x_data(df, is_fit=True, size=1.0):
#     target_size = (28, 28)
#     resized_images = []
#     for img in df['waferMap']:
#         resized_img = resize(img, target_size, anti_aliasing=True)
#         resized_images.append(resized_img)
#     _X_data = np.array(resized_images)
#     X_data = torch.tensor(_X_data, dtype=torch.float)
#     X_data = X_data.unsqueeze(1)
#     return X_data

def get_x_data(df, is_fit=True, size=1.0):
    target_size = (28, 28)
    resized_images = []
    for img in df['waferMap']:
        resized_img = resize(img, target_size, anti_aliasing=True)
        resized_images.append(resized_img)
        rotated_img_90 = np.rot90(resized_img, k=1, axes=(0, 1))
        rotated_img_180 = np.rot90(resized_img, k=2, axes=(0, 1))
        rotated_img_270 = np.rot90(resized_img, k=3, axes=(0, 1))

        resized_images.extend([rotated_img_90, rotated_img_180, rotated_img_270])

    _X_data = np.array(resized_images)
    X_data = torch.tensor(_X_data, dtype=torch.float)
    X_data = X_data.unsqueeze(1)
    return X_data

def solution(x_test_df, train_df):
    le = LabelEncoder()
    X_train = get_x_data(train_df)
    y_train = le.fit_transform(np.array(train_df['failureType']))
    X_test = get_x_data(x_test_df, False)

    transform = transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
        transforms.ToTensor(),
    ])

    # Normalize data
    normalize = transforms.Normalize(mean=[0.5], std=[0.5])
    X_train = normalize(X_train)
    X_test = normalize(X_test)

    # Create DataLoader with data augmentation
    train_dataset = TensorDataset(X_train, torch.tensor(y_train, dtype=torch.long).repeat(4)[:len(X_train)])

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    model = Net()
    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  # Adjust the learning rate
    criterion = nn.CrossEntropyLoss()

    # Early stopping parameters
    patience = 5  # Number of epochs with no improvement after which training will be stopped
    best_loss = float('inf')
    counter = 0

    epoch = 30  # Increase the number of epochs
    for i in range(1, epoch + 1):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            rotated_inputs = torch.rot90(inputs, k=1, dims=(2, 3))
            flipped_inputs = torch.flip(inputs, dims=(3,))
            rotated_inputs_180 = torch.rot90(inputs, k=2, dims=(2, 3))
            rotated_inputs_270 = torch.rot90(inputs, k=3, dims=(2, 3))

            augmented_inputs = torch.cat([inputs, rotated_inputs, flipped_inputs, rotated_inputs_180, rotated_inputs_270], dim=0)
            augmented_labels = torch.cat([labels] * 5, dim=0)

            optimizer.zero_grad()
            outputs = model(augmented_inputs)
            loss = criterion(outputs, augmented_labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch {i}/{epoch}, Loss: {loss.item()}")

        # Check for early stopping
        # if loss.item() < best_loss:
        #     best_loss = loss.item()
        #     counter = 0
        # else:
        #     counter += 1
        #     if counter >= patience:
        #         print(f"Early stopping at epoch {i} due to no improvement in {patience} epochs.")
        #         break

    # Move model back to CPU before predicting
    model.to("cpu")
    outputs = model(torch.tensor(X_test, dtype=torch.float))
    _, predicted = torch.max(outputs.data, 1)
    y_pred = le.inverse_transform(predicted.numpy())

    return pd.DataFrame({'failureType': y_pred}, index=x_test_df.index)

solution関数は以下のように活用され、平均精度を計算します。

In [16]:
t1 = time.time()

# データのインポート
df=pd.read_pickle("../input/LSWMD_25519.pkl")

# テスト用と学習用のデータを作成（テストする際は、random_stateの値などを編集してみてください）
train_df, test_df = train_test_split(df, stratify=df['failureType'], test_size=0.10, random_state=42)

y_test_df = test_df[['failureType']]
x_test_df = test_df.drop(columns=['failureType'])

# solution関数を実行
user_result_df = solution(x_test_df, train_df)

average_accuracy = 0
# ユーザーの提出物のフォーマット確認
if type(y_test_df) == type(user_result_df) and y_test_df.shape == user_result_df.shape:
    # 平均精度の計算
    accuracies = {}
    for failure_type in df['failureType'].unique():
        y_test_df_by_failure_type = y_test_df[y_test_df['failureType'] == failure_type]
        user_result_df_by_failure_type = user_result_df[y_test_df['failureType'] == failure_type]
        matching_rows = (y_test_df_by_failure_type == user_result_df_by_failure_type).all(axis=1).sum()
        accuracies[failure_type] = (matching_rows/(len(y_test_df_by_failure_type)))
    
    average_accuracy = sum(accuracies.values())/len(accuracies)

print(f"平均精度：{average_accuracy*100:.2f}%")
t2 = time.time()
print(f"実行時間：{t2-t1:.2f}秒")

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

labels = df['failureType'].unique()
cm = confusion_matrix(y_pred=user_result_df['failureType'], y_true=y_test_df['failureType'], labels=labels)
cmp = ConfusionMatrixDisplay(cm, display_labels=labels)
cmp.plot(cmap=plt.cm.Blues, xticks_rotation='vertical')

平均精度：11.94%
106 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
